# Predicting Price with Size, Location, and Neighborhood

In [11]:
pwd

'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\notebook'

## Library Importing

In [12]:
from glob import glob
import pandas as pd
import seaborn as sns #for data visualisation
from category_encoders import OneHotEncoder  #encoding categorical values
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact #for the interactive dashboard
from sklearn.impute import SimpleImputer #3imputting missing values
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401--automaed quality control code.
from sklearn.metrics import mean_absolute_error #to evaluate the model
from sklearn.pipeline import make_pipeline  #make the pipoeline
from sklearn.utils.validation import check_is_fitted #for unit testing

## Data Wrangling

In [37]:
#Loading the dataset with a function
def wrangle (filepath):
    #Load the CSV file into file path-local function applied
    df = pd.read_csv(filepath, encoding='ISO-8859-1') #read csv to the dataframe 
    #To display 'Capital Federal' only 
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    #To display 'Apartment' only 
    mask_apt = df["property_type"] == "apartment" 
    #To display price below'400,000'
    mask_price = df["price_aprox_usd"] < 400_000
    #Subset
    df = df[ mask_ba & mask_apt & mask_price]
    
    #Remove outliers by 'surface_area_m2' -----removing the 10th and 90th quantiles
    low = df["surface_covered_in_m2"].quantile(0.1)
    high = df["surface_covered_in_m2"].quantile(0.9)
    
    mask_area = df["surface_covered_in_m2"].between(low, high)
    
    df = df[mask_area]
    
    #split the lat-lon column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)
    
    #split place_with_parent_names
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)
    
    #drop features with high  null counts
    df.drop(columns=["floor","expenses"], inplace = True) 
    
    print('low = ', low, ';high = ', high)
    print('df.shape = ', df.shape)
    
    return df

In [38]:
#Collecting the files with the glob function
files = glob("C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\datasets/buenos-aires-real-estate-*.csv")
files


['C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\datasets\\buenos-aires-real-estate-1.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\datasets\\buenos-aires-real-estate-2.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\datasets\\buenos-aires-real-estate-3.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\datasets\\buenos-aires-real-estate-4.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\datasets\\buenos-aires-real-estate-5.csv']

In [39]:
#Loading the files with list comprehension to a df 
frames = [wrangle(file) for file in files]

EmptyDataError: No columns to parse from file

In [33]:
#type(frames[0]) #indicating its a list 

In [ ]:
frames[0].head() #dataframe 1

In [ ]:
#concatenate the dataframes 
df = pd.concat(frames, ignore_index = True)
df.head()

In [ ]:
 df.info()

In [ ]:
#shows percentage of representation of missing values more than 50% drop
#sum of null values in each column 0 rep false and 1 is true
df.isnull().sum()/len(df)

### High & low cardinality

In [ ]:
#View the datatype object
df.select_dtypes("object").head(50)

In [ ]:
#view the unique values within
df.select_dtypes("object").nunique()